## 1. contents loss

In [1]:
import numpy as np
from PIL import Image
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import keras
from keras import applications
from vgg16_avg import VGG16_Avg
from keras import metrics
from keras.models import Model
from keras import backend as K


import tensorflow as tf

work_dir = './src/Chapter08/'

content_image = Image.open(work_dir + 'bird_orig.png')

imagenet_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)


def subtract_imagenet_mean(image):
    return (image - imagenet_mean)[:, :, :, ::-1]


def add_imagenet_mean(image, s):
    return np.clip(image.reshape(s)[:, :, :, ::-1] + imagenet_mean, 0, 255)


vgg_model = VGG16_Avg(include_top=False)
content_layer = vgg_model.get_layer('block5_conv1').output
content_model = Model(vgg_model.input, content_layer)

content_image_array = subtract_imagenet_mean(np.expand_dims(np.array(content_image), 0))
content_image_shape = content_image_array.shape
target = K.variable(content_model.predict(content_image_array))

class ConvexOptimiser(object):
    def __init__(self, cost_function, tensor_shape):
        self.cost_function = cost_function
        self.tensor_shape = tensor_shape
        self.gradient_values = None

    def loss(self, point):
        loss_value, self.gradient_values = self.cost_function([point.reshape(self.tensor_shape)])
        return loss_value.astype(np.float64)

    def gradients(self, point):
        return self.gradient_values.flatten().astype(np.float64)


mse_loss = metrics.mean_squared_error(content_layer, target)
grads = K.gradients(mse_loss, vgg_model.input)
cost_function = K.function([vgg_model.input], [mse_loss]+grads)
optimiser = ConvexOptimiser(cost_function, content_image_shape)


def optimise(optimiser, iterations, point, tensor_shape, file_name):
    for i in range(iterations):
        point, min_val, info = fmin_l_bfgs_b(optimiser.loss, point.flatten(),
                                         fprime=optimiser.gradients, maxfun=20)
        point = np.clip(point, -127, 127)
        #print('Loss:', min_val)
        imsave(work_dir + 'gen_'+file_name+'_{i}.png', add_imagenet_mean(point.copy(), tensor_shape)[0])
    return point


def generate_rand_img(shape):
    return np.random.uniform(-2.5, 2.5, shape)/1
generated_image = generate_rand_img(content_image_shape)

iterations = 2
generated_image = optimise(optimiser, iterations, generated_image, content_image_shape, 'content')

Using TensorFlow backend.
/home/taeho/다운로드/DeepLearningforComputerVision_Code/vgg16_avg.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="block1_conv1", padding="same")`
  x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv1')(img_input)
/home/taeho/다운로드/DeepLearningforComputerVision_Code/vgg16_avg.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="block1_conv2", padding="same")`
  x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2')(x)
/home/taeho/다운로드/DeepLearningforComputerVision_Code/vgg16_avg.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="block2_conv1", padding="same")`
  x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1')(x)
/home/taeho/다운로드/DeepLearningforComputerVision_Code/vgg16_avg.py:55: U